<a href="https://colab.research.google.com/github/Purity-asiko/Pree-Projects/blob/main/Sentiment_Analysis_with_BERT_(IMDB_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Name: Sentiment Analysis with BERT (IMDB Dataset)

Goal: Classify movie reviews as positive or negative using BERT.

In [1]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
!pip install --quiet transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.datasets import imdb
import numpy as np

In [8]:
try:
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
    word_index = imdb.get_word_index()
    reverse_word_index = {v + 3: k for k, v in word_index.items()}
    def decode_review(encoded):
        return ' '.join([reverse_word_index.get(i, '?') for i in encoded])
    x_train_text = [decode_review(x) for x in x_train[:5000]]  # Limit for speed
    x_test_text = [decode_review(x) for x in x_test[:1000]]
    y_train = y_train[:5000]
    y_test = y_test[:1000]
except Exception as e:
    print("Error loading dataset:", e)

In [9]:
try:
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    def tokenize_reviews(reviews, max_length=128):
        return tokenizer(reviews, padding=True, truncation=True, max_length=max_length, return_tensors='tf')
    train_encodings = tokenize_reviews(x_train_text)
    test_encodings = tokenize_reviews(x_test_text)
except Exception as e:
    print("Error during tokenization:", e)

In [10]:
try:
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    model.fit(train_encodings['input_ids'], y_train, validation_data=(test_encodings['input_ids'], y_test),
              epochs=1, batch_size=8)  # Reduced for speed
except Exception as e:
    print("Error during training:", e)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error during training: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7a02d588c6d0>


In [11]:
try:
    test_loss, test_acc = model.evaluate(test_encodings['input_ids'], y_test)
    print(f"Test Accuracy: {test_acc}")
except Exception as e:
    print("Error during evaluation:", e)

Error during evaluation: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.
